In [ ]:
from sympy import *
import operator
import functools
import numpy as np
from itertools import combinations
init_printing()

In [14]:
class rooted_tree(object):
    """
    having dealt with tree atoms, now we manage the algebra system with "tables" of atoms and their coefficients
    these are perhaps best hash-mapped symbol atoms that can be manipulated in the algebra and then rendered as trees
    functions should also appear in the algebra and generate or consume atoms - i think some or all of these funtions can be defined on atoms?
    but then atoms are not a proper group element just a manipulation device - graft is one that would be on an atom algebra
    Now; rooted tree is a forest
    """
    def __init__(self,forest):
        if not isinstance(forest,list):forest=[forest]
        self._forest = forest   
        
    def html(self):    
        svg = ""
        for f in self._forest:svg += "<td style='vertical-align:top;background:white;padding:2px;margin:0'>{}</td>".format(f.html())      
        html = """<table color="red"> <tr>"""+svg+""" </tr></table>"""
        return svg

    def _repr_html_(self): 
        #return self._repr_latex_()
        return self.html()
    

class _tree_parser_(object):
    
    #__rmul__ = __mul__
    
    def __init__(self, in_degree_list):
        self.size = len(in_degree_list)
        self.__in_degree_list = np.array(in_degree_list)
        self.__parents_list = np.zeros(self.size, np.int)
        self.__subtree_size = np.zeros(self.size, np.int)
        self.__subtree_widths = np.zeros(self.size, np.int)
        self._visitor = -1
        self.__init_sub_trees__()
        self.__mark_widths__()
        self._styles = {}
        
    #todo tree factorial !    
        
    @property
    def grade (self):return self.size
    @property
    def parents(self):  return self.__parents_list
    @property
    def sub_tree_sizes(self):  return self.__subtree_size
    @property
    def sub_tree_widths(self):  return self.__subtree_widths
    @property
    def leaf_nodes(self):  return np.where(self.__subtree_size==1)[0]
    @property 
    def edges(self):return list(range(1,self.size))
    @property
    def admissable_cuts(self):
        """ 
            get all admissable cuts in the tree. put it on pandas to count things...
            import pandas as pd
            df=pd.DataFrame(list(tp.admissable_cuts)).fillna(0).astype(int)
            df["order"] = df.astype(bool).sum(axis=1)
            df
        """
        return self.__admissable_cuts__()
    
    @property
    def antipode(self):return self.__antipode__(self)
    
    def __mark_widths__(self):
        #mark the widths of the threes by counting all paths from leafs via nodes
        for l in self.leaf_nodes:
            parent = self.__parents_list[l]
            while parent != -1:
                self.__subtree_widths[parent]+= 1
                parent = self.__parents_list[parent]
    
    def __init_sub_trees__(self,pid=-1):
        self._visitor += 1
        self.__parents_list[self._visitor] = pid
        root = self._visitor 
        for i in range(self.__in_degree_list[self._visitor]):
            self.__init_sub_trees__(root)
        #however many nodes i visisted add to sub_tree(root) size
        self.__subtree_size[root] = self._visitor - root + 1

    ###Operations###

    def __add__(self, forest):return self.graft(forest)
        
    def graft(self,forest):  
        if not isinstance(forest,list): 
            forest = [forest]
        new_root_indegree = len(forest) + 1
        l = [new_root_indegree]
        l+= list(self.__in_degree_list)
        for t in forest:
            l += list(t.__in_degree_list)
        #trees completely determined by their depth-first-traverse of in-degrees
        return _tree_parser_(l)
       
    def __islands__(y):
        y_ext = np.r_[False,y==False, False]
        idx = np.flatnonzero(y_ext[:-1] != y_ext[1:])
        islands= zip(idx[:-1:2], idx[1::2])
        for i in islands: yield i[0], i[1] #return the data and the original offset

    def __mul__(self,b): return self.__rmul__(b)
    
    def __rmul__(self,b):
        a = self if isinstance(self,list) else [self]
        b = b if isinstance(b,list) else [b]
        return rooted_tree([*a,*b])
    
    def __antipode__(self,t):
        yield -1, t
        for c in t.admissable_cuts: 
            P,R = t.cut(c)
            yield -1, self.__antipode__(P)*R
        
    def __is_cut_admissable__(self, cut_set):
        for l in self.leaf_nodes:
            counter = 0
            if l in cut_set:counter+=1
            parent = self.__parents_list[l]
            while parent != -1:#walk to root
                #if any path we pass more than 1 edges in the cut set returned false
                if parent in cut_set:counter+=1
                if counter > 1:return false
                parent = self.__parents_list[parent]
        return True
    
    def __admissable_cuts__(self, order=None, complement=False):
        #by default consider all cuts below the root
        if order == None: order = self.size-1
        for o in range(1,order+1):
            for p in combinations(self.edges,r=o):
                if self.__is_cut_admissable__(list(p)) != complement: 
                    yield p
        
    def walk_down_from(self, i,v):
        v[i] = 0 # it us useful to record what we visit on all walks
        yield i
        for child in np.where(self.parents==i)[0]: 
            for c in self.walk_down_from(child,v): yield c
    
    def cut(self, cut_set):    
        P, head = [], np.array(list(self.__in_degree_list))
        mask = np.ones(len(head),np.bool)
        for c in cut_set: 
            #on the stored in degrees, decrement the one pointing to this sub tree
            head[self.__parents_list[c]]-=1
            #get all the nodes in the sub tree
            sub_tree_nodes = list(self.walk_down_from(c,mask))
            #make a new tree from their in-degrees
            P.append(_tree_parser_(self.__in_degree_list[sub_tree_nodes]))
        #create a new head tree from the stuff still in the tree using udpated in-degrees
        R = _tree_parser_(head[mask])
        return P,R
        
    def get_coordinates(self):
        lateral_cursor = np.zeros(self.size, np.int)

        def __walk__(i,parent=-1,my_index=0,depth=0,latCur=lateral_cursor,parent_offset=0):
            layer = np.where(self.__parents_list==i)[0]
            parent_width = self.__subtree_widths[parent] if parent != -1 else 0
            width = self.__subtree_widths[i] if self.__subtree_widths[i] > 0 else 1        
            offset = lateral_cursor[parent]
            lateral_cursor[parent] += width     
            X = parent_offset + offset - parent_width/2. + width/2.
            #print("**",parent_offset , offset - parent_width/2. ,width/2.,"**")
            #print("me", i, "my index", my_index, "my_offset", my_offset, "parent" ,parent, "mywidth", width, "parent width", self.__subtree_widths[parent])
            #print(i, "parent offset-myoffset", parent_offset, X, "mywidth", width, "lateral offset", offset)
            yield [X, depth]
            
            for index, child in enumerate(layer): 
                for c in __walk__(child,i,index,depth+1,lateral_cursor,X):  yield c
                    
        return list(__walk__(0,depth=0))
    
    def __repr__(self):return self._repr_latex_()
    
    @property
    def value(self):  return Symbol("\\bullet_{\{ "+str(42)+" \}}",commutative=False)
    def __call__(self): return self.value
    def _sympy_(self):return self.__call()

    def with_style(self, style={}): 
        self._styles = style
        return self
        
    def html(self):    
        def __draw_line__(pts,colour="black"): return """<line x1="{0}" y1="{1}" x2="{2}" y2="{3}" stroke-width:1" stroke="{4}" />""".format(*pts,colour)
        buffer,margin,scale,body = 3,  (0,5),(10,20),""
        coords = np.array([ [c[0]*scale[0]+margin[0], c[1]*scale[1]+margin[1]] for c in self.get_coordinates()])
        bounds = (coords[:,0].max()+buffer,coords[:,1].max()+buffer)
        
        for c in coords: body += """<circle cx="{}" cy="{}" r="2" stroke="black" stroke-width="1" fill="black" /> """.format(*c)
        for i,d in enumerate(self.__parents_list):
            colour = "black"
            if "edge_colours" in self._styles and i in self._styles["edge_colours"]: 
                colour = self._styles["edge_colours"][i]
            if i != 0:body += __draw_line__(list(coords[i]) + list(coords[d]),colour=colour)       
        
        svg =  """<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="{1}" y="{2}" width="{3}" height="{4}">
               <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> {0}  </g> </svg>""".format(body, 0,0, bounds[0], bounds[1])
        
        return svg

    def _repr_html_(self): 
        #return self._repr_latex_()
        return self.html()

    def _repr_latex_(self):  return latex(self.value,mode='inline')
    
    #''helper
    
    def make_mark_cut_style(cut):
        """
        example use: tp.with_style(_tree_parser_.make_mark_cut_style([1,6,8]))
        """
        style = {"edge_colours":{}}
        for c in cut: style["edge_colours"][c] = "grey"
        return style

tp = _tree_parser_([3,2,0,2,0,0,1,0,0])     
tp

$\bullet_{\{ 42 \}}$

In [15]:
#tp.with_style(_tree_parser_.make_mark_cut_style([1,6,8]))

In [16]:
tp2 = _tree_parser_([1,0])     
tp2

$\bullet_{\{ 42 \}}$

In [17]:
tp.graft([tp2])

$\bullet_{\{ 42 \}}$

In [18]:
tp + tp2

$\bullet_{\{ 42 \}}$

In [19]:
tp * tp2

In [20]:
list(tp.admissable_cuts)

## example cuts: P * R

In [21]:
P,R=tp.cut([2,6])
#R is the single tree at the end, P is the forest in front
rooted_tree([*P,R])

In [22]:
P,R=tp.cut([2,4,8])
#R is the single tree at the end, P is the forest in front
rooted_tree([*P,R])

In [23]:
P,R=tp.cut((2,4,5,7,8))
#R is the single tree at the end, P is the forest in front
rooted_tree([*P,R])

In [24]:
rooted_tree([tp,tp2,tp2,])

In [25]:
#create a table of admissable cuts => P R

In [ ]:
from sympy.physics.quantum import TensorProduct

A= TensorProduct(Symbol("\mathbb{1}",commutative=False), Symbol("\mathbb{1}",commutative=False))

#B = TensorProduct(rooted_tree(5), rooted_tree(7))
rooted_tree(5) * rooted_tree(5)

In [ ]:
#SOME EXAMPLES OF SYB CLASSES THAT SEEMED TO WORK

# A = Symbol('A',commutative=True)

# B = Symbol('B',commutative=True)
# C = Symbol('C',commutative=False)
# D = Symbol('D',commutative=False)

# V = TensorProduct(A,B) + TensorProduct(C,D)
# V

# import sympy as sp
# from sympy.core.expr import Expr

# class Tensor(Expr):
#     __slots__ = ['is_commutative']

#     def __new__(cls, l, r):
#         l = sp.sympify(l)
#         r = sp.sympify(r)
#         obj = Expr.__new__(cls, l, r)
#         obj.is_commutative = False
#         return obj
    
#     def _print(self, e=None):return self._display_()
#     def __srepr__(self):  return self._display_()
#     def __repr__(self):  return self._display_()
    
#     def _display_(self):    
#         body ="""<circle cx="20" cy="20" r="3" stroke="black" stroke-width="1" fill="black" /> """
#         return  """<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="{1}" y="{2}" width="500" height="120">
#       <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> {0}  </g> </svg>""".format(body, 20,20)
#         #return latex("bullet",mode='inline')

        
#     def _repr_latex_(self):    
#         body ="""<circle cx="20" cy="20" r="3" stroke="black" stroke-width="1" fill="black" /> """
#         return  """<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="{1}" y="{2}" width="500" height="120">
#       <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> {0}  </g> </svg>""".format(body, 20,20)
#         #return latex("bullet",mode='inline')

#     def __neg__(self):
#         return self

#     def __mul__(self, other):
#         if isinstance(other, Tensor):
#             return Tensor(self.args[0] * other.args[0], other.args[1] * self.args[1])
#         elif other.is_number:
#             if other % 2 == 0:
#                 return 0
#             else:
#                 return self
#         else:
#             return sp.Mul(self, other)


# x, y = sp.symbols('x, y', commutative=False)

# Ym = Tensor(y, 1) - Tensor(1, y)
# Yp = Tensor(y, 1) + Tensor(1, y)
# Xm = Tensor(x, 1) - Tensor(1, x)

# d1 = Ym * Yp + Xm * 0
# # print(d1)
# # print(sp.expand(d1))

# # d2 = Xm * Ym
# # print(d2)
# # print(sp.expand(d2))
# #TensorProduct(Tensor(y, 1),Tensor(y, 1))